<a href="https://colab.research.google.com/github/fxn-m/design-project-4/blob/main/methods/method_2/train_yolov5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# automate jupyter/colab config
def isnotebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
            
    except NameError:
        return False      # Probably standard Python interpreter
    
if isnotebook():
  print('Jupyter Notebook')
else:
  print('Colab')

In [ ]:
# colab 
if not isnotebook():
    from tensorflow.python.client import device_lib
    print(device_lib.list_local_devices())
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
# colab 
if not isnotebook():    
    %cd /content/drive/MyDrive/University/year_4/dp4/working-cells
    !rm -r /content/drive/MyDrive/University/year_4/dp4/working-cells/yolov5
    !git clone https://github.com/ultralytics/yolov5 # clone 
    
    %cd yolov5
    %pip install -qr requirements.txt # install dependencies

    import torch
    import sys
    sys.path.insert(0, './yolov5')

    %cd /content/drive/MyDrive/University/year_4/dp4/working-cells

    from yolov5 import utils, val
    display = utils.notebook_init() #check

    %pip install -q roboflow
    %pip install -q wandb 

    import tensorflow as tf
    tf.test.gpu_device_name()
    
# jupyter
else: 
    %cd working-cells
        
    import shutil, stat

    def on_rm_error(func, path, exc_info):
        # path contains the path of the file that couldn't be removed
        # let's just assume that it's read-only and unlink it.
        os.chmod(path, stat.S_IWRITE )
        os.unlink(path)

    shutil.rmtree('yolov5', onerror = on_rm_error)
    
    !git clone https://github.com/ultralytics/yolov5
        
    %cd yolov5
    %pip install -qr requirements.txt 

    import torch

    %cd ..
    from yolov5 import utils, val
    display = utils.notebook_init() #check
    
    %cd ..

In [ ]:
# import libraries
import os
from IPython.display import Image, clear_output
import wandb
import matplotlib.pyplot as plt
import sklearn as skl
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from roboflow import Roboflow

tqdm.pandas()

model = 'yolov5s6'
dim = 640
batch = 12
epochs = 150
opt = 'Adam'
module = 'module_1'
tl = 'Y'
mos = 'Y'

name = f'{module}-dim{dim}-batch_size{batch}-tl{tl}-mos{mos}-colab' 

project = 'YOLOv5-cells'

In [ ]:
%cd /content/drive/MyDrive/University/year_4/dp4/secrets
  
# WandB - credentials & login
with open("wandbkey.txt", 'r') as k:

    try:
        wandb.login(key=k.read())
        anonymous = None
    except:
        wandb.login(anonymous='must')

# roboflow - credentials
with open("rbfkey2.txt", 'r') as k:
    rbfkey = k.read()

In [ ]:
%cd /content/drive/MyDrive/University/year_4/dp4/working-cells/yolov5/

rf = Roboflow(api_key=rbfkey)
project = rf.workspace("yolov5moduleod").project("modules_1-3")
dataset = project.version(1).download("yolov5")

location = dataset.location

In [ ]:
dataset.location

# hyperparameter sweep

In [ ]:
# %%writefile /content/drive/MyDrive/University/year_4/dp4/working-cells/yolov5/utils/loggers/wandb/sweep.yaml

# program: /content/drive/MyDrive/University/year_4/dp4/working-cells/yolov5/utils/loggers/wandb/sweep.py

# method: random

# metric:
# name: metrics/mAP_0.5
# goal: maximize

# entity: fxn-m

# project: YOLOv5-cells-sweeps

# parameters:

# batch_size:
# distribution: int_uniform 
# max: 30
# min: 10

# optimizer:
# value: Adam

# weights:
# value: yolov5s6.pt

# epochs:
# value: 150

# imgsz:
# value: 640

# data:
# value: /content/drive/MyDrive/University/year_4/dp4/working-cells/yolov5/modules_1-3-1/data.yaml

# warmup_momentum:
# max: 0.9
# min: 0.7     
# warmup_bias_lr:
# max: 0.2
# min: 0.01
# warmup_epochs:
# max: 4
# min: 2
# weight_decay:
# max: 0.001
# min: 0.0001
# fl_gamma: 
# max: 1
# min: 0
# translate:
# max: 0.2
# min: 0.05
# momentum:
# max: 0.99
# min: 0.8
# anchor_t:
# max: 6
# min: 2
# anchors:
# max: 4
# min: 2
# obj_pw:
# max: 2.0
# min: 0.2
# mosaic:
# value: 0.5
# flipud:
# value: 0.5
# fliplr:
# value: 0.5
# cls_pw:
# max: 2.0
# min: 0.2
# scale:
# max: 1.00
# min: 0.25
# mixup:
# value: 0
# iou_t:
# max: 0.5
# min: 0.2
# hsv_v:
# max: 0.6
# min: 0.2
# hsv_s:
# max: 0.8
# min: 0.5
# hsv_h:
# max: 0.1
# min: 0.01      
# degrees:
# value: 0
# translate:
# max: 0.2
# min: 0.0
# scale:
# max: 1
# min: 0
# shear:
# value: 0
# perspective:
# max: 0.001
# min: 0.0
# obj:
# max: 2.0
# min: 0.2
# lrf:
# max: 0.15
# min: 0.05
# lr0:
# max: 0.01 
# min: 0.001
# cls:
# max: 1.0
# min: 0.1
# box:
# max: 0.1
# min: 0.01
# copy_paste:
# value: 0

In [ ]:
# !wandb sweep /content/drive/MyDrive/University/year_4/dp4/working-cells/yolov5/utils/loggers/wandb/sweep.yaml

In [ ]:
# !wandb agent fxn-m/YOLOv5-cells-sweeps/kyg9c3ej

# set hyperparameters

In [ ]:
# %%writefile /content/drive/MyDrive/University/year_4/dp4/working-cells/yolov5/hyp.yaml

# lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
# lrf: 0.01  # final OneCycleLR learning rate (lr0 * lrf)
# momentum: 0.937  # SGD momentum/Adam beta1
# weight_decay: 0.0005  # optimizer weight decay 5e-4
# warmup_epochs: 3.0  # warmup epochs (fractions ok)
# warmup_momentum: 0.8  # warmup initial momentum
# warmup_bias_lr: 0.1  # warmup initial bias lr
# box: 0.05  # box loss gain
# cls: 0.5  # cls loss gain
# cls_pw: 1.0  # cls BCELoss positive_weight
# obj: 1.0  # obj loss gain (scale with pixels)
# obj_pw: 1.0  # obj BCELoss positive_weight
# iou_t: 0.20  # IoU training threshold
# anchor_t: 4.0  # anchor-multiple threshold
# # anchors: 3  # anchors per output layer (0 to ignore)
# fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
# hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
# hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
# hsv_v: 0.4  # image HSV-Value augmentation (fraction)
# degrees: 0.0  # image rotation (+/- deg)
# translate: 0.1  # image translation (+/- fraction)
# scale: 0.5  # image scale (+/- gain)
# shear: 0.0  # image shear (+/- deg)
# perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
# flipud: 0.0  # image flip up-down (probability)
# fliplr: 0.5  # image flip left-right (probability)
# mosaic: 0  # image mosaic (probability)
# mixup: 0.0  # image mixup (probability)
# copy_paste: 0.0  # segment copy-paste (probability)

# optimizer:
#   desc: null
#   value: Adam

# weights:
#   desc: null
#   value: yolov5s6.pt


In [ ]:
# hyps = '/content/drive/MyDrive/University/year_4/dp4/working-cells/yolov5/hyp.yaml'

# standard train

In [ ]:
%cd /content/drive/MyDrive/University/year_4/dp4/working-cells/yolov5/

!python train.py --img 640 --batch 12 --epochs 150 --optimizer {opt} --data {location}/data.yaml --weights yolov5s6.pt --project 'YOLOv5-cells' --name {name}

# !python train.py --img {dim} --batch {batch} --epochs {epochs} --optimizer {opt} --data {location}/data.yaml --hyp {hyps} --weights {model}.pt --cache --project 'YOLOv5-cells' --name {name} --patience 0

# !python train.py --img {dim} --batch {batch} --epochs {epochs} --optimizer {opt} --data {location}/data.yaml --weights '' --cfg yolov5s.yaml --cache --project 'YOLOv5-cells' --name {name} --patience 0

# dataset size experiments

In [ ]:
model = "yolov5s6"
print(dim, batch, epochs)

In [ ]:
for epochs in np.linspace(10, 12, 11):
  print(int(epochs))

In [ ]:
for epochs in np.linspace(10, 12, 11):

  %cd /content/drive/MyDrive/University/year_4/dp4/working-cells/yolov5/

  epochs = int(epochs)

  !python train.py --img {dim} --batch {batch} --epochs {epochs} --optimizer {opt} --data {location}/data.yaml --weights {model}.pt --cache --project 'YOLOv5-cells' --name {name} --patience 0

# evaluate model

In [ ]:
%cd /content/drive/MyDrive/University/year_4/dp4/working-cells/yolov5

path = './YOLOv5-cells/module_1-dim640-batch_size12-tlNO-mosNO-colab2/weights/best.pt'

!python detect.py --weights {path} --img {dim} --conf 0.3 --source {dataset.location}/valid/images

In [ ]:
#display inference on test images

import glob
from IPython.display import Image, display

display_width = 2048/4
display_height= 1536/4



for imageName in glob.glob('/content/drive/MyDrive/University/year_4/dp4/working-cells/yolov5/runs/detect/exp6/*.jpg'): #assuming JPG
    # display(Image(filename=imageName))
    display(Image(filename=imageName, width=display_width, height=display_height))
    print("\n")